In [1]:
import pandas as pd
import random
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
x = random.sample(range(1,20), 10)
d = pd.DataFrame(x, columns=['x'])
# d = pd.DataFrame({'x': [1, 2, 3, 4, 5],
#                    'y': [1., 2., 3., 4., 5.]})

In [3]:
df = dd.from_pandas(d, npartitions=2)

In [4]:
def our_add(df):
    return df + 100

In [5]:
new_df = df.x.map_partitions(lambda a: a+100)

In [6]:
new_df.compute()

0    102
1    111
2    113
3    105
4    116
5    104
6    112
7    108
8    118
9    119
Name: x, dtype: int64

In [47]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                   'y': [1., 2., 3., 4., 5.]})
ddf = dd.from_pandas(df, npartitions=2)

In [48]:
ddf.compute()

,x,y
0,1,1.0
1,2,2.0
2,3,3.0
3,4,4.0
4,5,5.0


In [51]:
def myadd(df, b):
    return df + b
res = ddf.x.map_partitions(myadd, b=2)

In [52]:
res.compute()

0    3
1    4
2    5
3    6
4    7
Name: x, dtype: int64

In [121]:
lis = ['a/b/c/d.txt', 'e/f/g/h/i.txt']
d = pd.DataFrame(lis, columns=['x'], dtype=np.str)
df = dd.from_pandas(d, npartitions=2)

/tmp/ipykernel_305896/118707986.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  d = pd.DataFrame(lis, columns=['x'], dtype=np.str)


In [124]:
def your_add(df):
    return df.split("/")[-1]

In [125]:
new_df = df.x.map(your_add)

In [126]:
new_df.compute()

0    d.txt
1    i.txt
Name: x, dtype: object

In [7]:
ddf = dd.from_pandas(
    pd.DataFrame({'image_path': ['x.jpg'] * 10, 'label': range(1,11)}),
    npartitions=2
)

In [8]:
ddf.compute()

,image_path,label
0,x.jpg,1
1,x.jpg,2
2,x.jpg,3
3,x.jpg,4
4,x.jpg,5
5,x.jpg,6
6,x.jpg,7
7,x.jpg,8
8,x.jpg,9
9,x.jpg,10


In [9]:
def preprocess(path):
    return len(path)

In [10]:
sum_col1 = ddf.image_path.apply(lambda x: preprocess(str(x)), meta=('sum_col1', int))
sum_col1.head()

0    5
1    5
2    5
3    5
4    5
Name: sum_col1, dtype: int64

In [11]:
sum_col2 = ddf.image_path.map_partitions(lambda x: preprocess(str(x)), meta=('sum_col2', int))

In [12]:
sum_col2.compute()

0    86
1    86
dtype: int64

In [ ]:
sum_col3 = ddf.image_path.map_partitions(
    lambda x: x.apply(lambda y: preprocess(str(y))),
    meta=('sum_col3', int)
)